<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [18]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Gabriel Henrique Baca Rado, 21012865' #@param {type:"string"}






In [19]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [21]:
host_counts = df['host_id'].value_counts()
print(host_counts.head(5))

display(df[df['host_id']==219517861].host_name.head(1))
display(df[df['host_id']==120762452].host_name.head(1))
display(df[df['host_id']==30283594].host_name.head(1))

host_id
219517861    7
120762452    4
30283594     4
19303369     4
18996093     2
Name: count, dtype: int64


676    Sonder (NYC)
Name: host_name, dtype: object

402    Stanley
Name: host_name, dtype: object

241    Kara
Name: host_name, dtype: object

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [22]:
hosts_um = host_counts[host_counts == 1]
hosts_mais_de_um = host_counts[host_counts > 1]

preco_mais_um_imovel = df[df['host_id'].isin(hosts_um.index)].price.mean()
preco_mais_de_um_imovel = df[df['host_id'].isin(hosts_mais_de_um.index)].price.mean()

print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [23]:
# Calculate Q1, Q3, and IQR for prices of all properties
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Define the outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter Manhattan properties
manhattan_df = df[df['neighbourhood_group'] == 'Manhattan']

# Count Manhattan outliers
manhattan_outliers = manhattan_df[
    (manhattan_df['price'] < lower_bound) | (manhattan_df['price'] > upper_bound)
]
num_manhattan_outliers = len(manhattan_outliers)

# Calculate the percentage of Manhattan outliers
perc_Manhattan = (num_manhattan_outliers / len(manhattan_df)) * 100

print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')

Percentuais de outliers de preços, Manhattan: 10.05 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [24]:
# Create the 'word' column based on the presence of keywords in the 'name' column
def categorize_description(name):
    name = str(name).lower()
    if 'beautiful' in name:
        return 'beautiful'
    elif 'sunny' in name:
        return 'sunny'
    elif 'spacious' in name:
        return 'spacious'
    elif 'heart' in name:
        return 'heart'
    else:
        return 'others'

df['word'] = df['name'].apply(categorize_description)

# Group by the 'word' column and calculate the count and mean price for each group
word_stats = df.groupby('word')['price'].agg(['count', 'mean'])

word_stats


count        mean
word                        
beautiful     41  137.926829
heart         36  166.388889
others       624  157.886218
spacious      60  128.516667
sunny         43  137.674419

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [28]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

# Merge the two dataframes based on the 'id' column
df = pd.merge(df, df_taxes, on='id', how='left')

# Fill NaN values in the 'value' column with 0
df['value'] = df['value'].fillna(0)

id                                               name    host_id  \
0        20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
1        20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
2        25235                  Large 2 Bedroom Great for Groups!      87773   
3        25235                  Large 2 Bedroom Great for Groups!      87773   
4        76761     Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm     118971   
...        ...                                                ...        ...   
1563  36439512                       The Bushwick Backhouse Loft.  180336853   
1564  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
1565  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
1566  36472171                      1 bedroom in sunlit apartment   99144947   
1567  36472171                      1 bedroom in sunlit apartment   99144947   

       host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
2         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
3         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
4         Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
...          ...                 ...                 ...       ...        ...   
1563        Dash            Brooklyn            Bushwick  40.70470  -73.92250   
1564     Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
1565     Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
1566      Brenda           Manhattan              Inwood  40.86845  -73.92449   
1567      Brenda           Manhattan              Inwood  40.86845  -73.92449   

            room_type  price  minimum_nights  number_of_reviews last_review  \
0     Entire home/apt    100               5                168  2018-07-22   
1     Entire home/apt    100               5                168  2018-07-22   
2     Entire home/apt    125              90                162  2019-06-28   
3     Entire home/apt    125              90                162  2019-06-28   
4     Entire home/apt    169               2                398  2019-06-28   
...               ...    ...             ...                ...         ...   
1563     Private room     70               2                  0         NaN   
1564     Private room    800               1                  0         NaN   
1565     Private room    800               1                  0         NaN   
1566     Private room     80               1                  0         NaN   
1567     Private room     80               1                  0         NaN   

      reviews_per_month  calculated_host_listings_count  availability_365  \
0                  1.57                               1                 0   
1                  1.57                               1                 0   
2                  1.46                               2               137   
3                  1.46                               2               137   
4                  3.97                               3               182   
...                 ...                             ...               ...   
1563                NaN                               1               255   
1564                NaN                               1                23   
1565                NaN                               1                23   
1566                NaN                               1                79   
1567                NaN                               1                79   

        word   type   value  
0     others  taxes   789.0  
1     others  fines   796.0  
2     others  taxes  1779.0  
3     others  fines     0.0  
4     others  taxes  1516.0  
...      ...    ...     

In [35]:
df.columns.to_list()

columns = ['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'word']

new_df = df.pivot_table(index=columns,columns="type",values='value').reset_index()
new_df.head()

type      id                                            name  host_id  \
0      20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1      25235               Large 2 Bedroom Great for Groups!    87773   
2      76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3     135465             Garden apartment close to Manhattan   663879   
4     219482                        A Cozy Brooklyn Hideaway   759583   

type    host_name neighbourhood_group       neighbourhood  latitude  \
0      Christiana            Brooklyn        Williamsburg  40.70984   
1          Shelly            Brooklyn  Bedford-Stuyvesant  40.68373   
2          Evelyn            Brooklyn         South Slope  40.66552   
3     Christopher            Brooklyn         Fort Greene  40.68626   
4            Pepe            Brooklyn        Clinton Hill  40.68275   

type  longitude        room_type  price  minimum_nights  number_of_reviews  \
0     -73.95775  Entire home/apt    100               5                168   
1     -73.92377  Entire home/apt    125              90                162   
2     -73.99019  Entire home/apt    169               2                398   
3     -73.97598  Entire home/apt    170               3                 86   
4     -73.96148  Entire home/apt    172               4                  8   

type last_review  reviews_per_month  calculated_host_listings_count  \
0     2018-07-22               1.57                               1   
1     2019-06-28               1.46                               2   
2     2019-06-28               3.97                               3   
3     2019-06-20               0.91                               2   
4     2017-10-14               0.09                               2   

type  availability_365    word   fines   taxes  
0                    0  others   796.0   789.0  
1                  137  others     0.0  1779.0  
2                  182  others  1935.0  1516.0  
3                  286  others     0.0     0.0  
4                  189  others     0.0     0.0

In [38]:
# prompt: Qual o preço médio de Taxes e Fines para o Bairro do Manhattan? usando o new_df

# Calculate the average taxes and fines for Manhattan
manhattan_df = new_df[new_df['neighbourhood_group'] == 'Manhattan']
average_taxes_manhattan = manhattan_df['taxes'].mean()
average_fines_manhattan = manhattan_df['fines'].mean()
print(f"O preço médio de Taxes para o Bairro do Manhattan é: {average_taxes_manhattan}")
print(f"O preço médio de Fines para o Bairro do Manhattan é: {average_fines_manhattan}")


O preço médio de Taxes para o Bairro do Manhattan é: 873.5
O preço médio de Fines para o Bairro do Manhattan é: 307.8296296296296


# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [42]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 26517


#### Aqui um *sample* do report.

In [90]:
# Calculate the mean price for each neighbourhood_group
mean_prices = df.groupby('neighbourhood_group')['price'].mean()

# Create an empty list to store the report data
report_data = []

# Initialize page and line counters
page_id = 1
line_id = 1

# Iterate through the neighbourhood_groups
for neighbourhood_group in sorted(df['neighbourhood_group'].unique()):
    # Filter data for the current neighbourhood_group
    group_data = df[df['neighbourhood_group'] == neighbourhood_group].sort_values(by='neighbourhood')

    # Iterate through the neighbourhoods within the group
    for neighbourhood in sorted(group_data['neighbourhood'].unique()):
        # Filter data for the current neighbourhood
        neighbourhood_data = group_data[group_data['neighbourhood'] == neighbourhood].sort_values(by='neighbourhood')

        # Add each row to the report
        for index, row in neighbourhood_data.iterrows():
            report_data.append([
                row['neighbourhood_group'],
                row['neighbourhood'],
                row['price'],
                page_id,
                line_id
            ])
            line_id += 1
            if line_id > 30:
                page_id += 1
                line_id = 1

    # Add the mean price row for the neighbourhood_group
    report_data.append([
        neighbourhood_group,
        'MEAN',
        mean_prices[neighbourhood_group],
        page_id,
        line_id
    ])
#    line_id += 1
#    if line_id > 30:
    page_id += 1
    line_id = 1

# Create the report DataFrame
df_report = pd.DataFrame(report_data, columns=['neighbourhood_group', 'neighbourhood', 'price', 'page_id', 'line_id'])

# Sort the report DataFrame
#df_report = df_report.sort_values(by=['neighbourhood_group', 'neighbourhood'])

# Calculate the sum of prices for specified pages
sum_price_pages = df_report[df_report.page_id.isin([1, 24, 25, 28])].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')


A soma dos preços das páginas 1, 24, 25 e 28 é: 13227


In [93]:
df_report

neighbourhood_group      neighbourhood       price  page_id  line_id
0                  Bronx            Belmont   29.000000        1        1
1                  Bronx            Belmont   29.000000        1        2
2                  Bronx  Claremont Village   67.000000        1        3
3                  Bronx  Claremont Village   67.000000        1        4
4                  Bronx          Concourse   40.000000        1        5
...                  ...                ...         ...      ...      ...
1568       Staten Island         St. George  130.000000       56       10
1569       Staten Island          Todt Hill  135.000000       56       11
1570       Staten Island          Todt Hill  135.000000       56       12
1571       Staten Island      Tompkinsville   59.000000       56       13
1572       Staten Island               MEAN   91.769231       56       14

[1573 rows x 5 columns]

- 1. Coleta de Dados: O código inicia com a leitura de dois arquivos CSV de um repositório online (GitHub).
- 2. Limpeza de Dados: Tratamento de valores faltantes: No código, a coluna 'value' do DataFrame df é preenchida com 0 onde há valores NaN.
- 3. Integração de Dados: Os dois dataframes são combinados usando a coluna 'id' como chave.
- 4. Redução de Dados: Seleção de atributos:  O código seleciona algumas colunas para usar no pivot table.
- 5. Transformação de Dados: Criação da coluna `word` categorizando descrições dos imóveis.
- 6. Formatação de Dados: Esta etapa é crucial para preparar os dados para a etapa de modelagem ou para visualização.